In [1]:
import os
from glob import glob

files = [y for x in os.walk('E:/PDAL/doc/stages') for y in glob(os.path.join(x[0], '*.rst'))]

In [2]:
names = [os.path.basename(file).rsplit('.rst')[0] for file in files]

In [3]:
reader_names = [name.split('.')[1] for name in names if 'readers' in name and '.' in name]
writer_names = [name.split('.')[1] for name in names if 'writers' in name and '.' in name]
filter_names = [name.split('.')[1] for name in names if 'filters' in name and '.' in name]

In [4]:
def auto_func(name,options,helptext,module):
    lines = [
        f"def {name}({'=None, '.join(options)}):",
        r'    """' + helptext + r'    """',
        '',
        '    vars = dict()',
        f"    vars['type'] = '{module}.{name}'",
        "    vars.update(locals().copy())",
        "    del(vars['kwargs'])",
        "    del(vars['vars'])",
        "    for key in list(vars.keys()):",
        "        if vars[key] is None:",
        "            del(vars[key])",
        "    vars.update(kwargs)",

        "    return vars",
        ''
    ]

    return lines

In [5]:
def auto_module(module,names):
    lines = list()

    for name in names:

        helpfile = 'E:/PDAL/doc/stages/' + module + '.' + name + '.rst'

        try:
            with open(helpfile) as f:
                helptext = f.readlines()

            readoptions = False
            options = list()

            for line, nextline in zip(helptext,helptext[1:]):
                if readoptions==True:
                    line = line.strip()
                    line_strip_sc = line.strip("'-:[]=+()%^$#!&*<>.,/`{}|~")
                    if len(line)>0 and len(line.split(' '))==1 and len(line)==len(line_strip_sc) and line!='class' and line != 'as' and line != 'global' and '|' not in line:
                        options.append(line)

                if 'Options' in line and '----' in nextline:
                    readoptions = True


            helptext = '\n'.join(helptext)
            options.extend(['inputs','tag','**kwargs'])

        except:
            helptext = ''
            options = ['**kwargs']

        lines.extend(auto_func(name,options,helptext,module))

    with open(module + '.py','w') as f:
        f.writelines('\n'.join(lines))

In [6]:
auto_module('readers',reader_names)
auto_module('writers',writer_names)
auto_module('filters',filter_names)

In [7]:
import filters
import writers
import readers

In [8]:
readers.e57(filename='E:/SanCarlos/TLS_1/San_carlos_1_0.e57')

{'type': 'readers.e57', 'filename': 'E:/SanCarlos/TLS_1/San_carlos_1_0.e57'}